### 用 tf.data 加载 CSV 数据

In [3]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

#### 内存数据

小型CSV数据集使用Pandas Dataframe 或 NumPy 数组加载到内存中。

做一个回归模型来预测年龄

In [4]:
abalone_train = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

abalone_train.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965,7
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250,6
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700,14
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600,16
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300,13


In [5]:
abalone_features = abalone_train.copy()
# 数据集的名义任务是根据其他测量值预测年龄，因此将特征和标签(Age)分开以进行训练。
abalone_labels = abalone_features.pop('Age')

In [9]:
abalone_features = np.array(abalone_features)
abalone_features

array([[0.435, 0.335, 0.11 , ..., 0.136, 0.077, 0.097],
       [0.585, 0.45 , 0.125, ..., 0.354, 0.207, 0.225],
       [0.655, 0.51 , 0.16 , ..., 0.396, 0.282, 0.37 ],
       ...,
       [0.53 , 0.42 , 0.13 , ..., 0.374, 0.167, 0.249],
       [0.395, 0.315, 0.105, ..., 0.118, 0.091, 0.119],
       [0.45 , 0.355, 0.12 , ..., 0.115, 0.067, 0.16 ]])

In [10]:
abalone_model = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
])

abalone_model.compile(loss=tf.losses.MeanSquaredError(),
                      optimizer=tf.optimizers.Adam())

Metal device set to: Apple M1


2021-08-22 03:59:47.407830: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-22 03:59:47.408930: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
abalone_model.fit(abalone_features, abalone_labels, epochs=10)

2021-08-22 03:59:51.561326: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-22 03:59:51.565316: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-08-22 03:59:51.731927: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
104/104 [==============================] - 1s 5ms/step - loss: 47.1071
Epoch 2/10
104/104 [==============================] - 0s 4ms/step - loss: 9.4238

#### 基本预处理

In [ ]:
# todo 归一化一章
normalize = preprocessing.Normalization()

normalize.adapt(abalone_features)

In [ ]:
norm_abalone_model = tf.keras.Sequential([
    normalize,
    layers.Dense(64)
])

#### 混合数据类型

“泰坦尼克号”数据集包含有关泰坦尼克号乘客的信息。该数据集的名义任务是预测谁幸存下来。

数据中包含字符串，数字等，需要进行预处理

In [12]:
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")

titanic.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [13]:
titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')

In [14]:
input = tf.keras.Input(shape=(), dtype=tf.float32)
result = 2 * input + 1

calc = tf.keras.Model(inputs=input, outputs=result)

In [17]:
print(calc(2).numpy())

5.0


In [30]:
# 列名称和 dtype
inputs = {}

for name, column in titanic_features.items():
    dtype = column.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32
    inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'sex')>,
 'age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'age')>,
 'n_siblings_spouses': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'n_siblings_spouses')>,
 'parch': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'parch')>,
 'fare': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fare')>,
 'class': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'class')>,
 'deck': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'deck')>,
 'embark_town': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'embark_town')>,
 'alone': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'alone')>}

In [34]:
# 将数字输入连接在一起，并通过规范化层运行它们：
numeric_inputs = {name: input
                   for name, input in inputs.items()
                   if input.dtype == tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()

norm.adapt(np.array(titanic[numeric_inputs.keys()]))

all_numeric_inputs = norm(x)

all_numeric_inputs

2021-08-22 04:48:21.519408: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-22 04:48:21.531033: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization_1')>

In [ ]:
preprocessing_inputs = [all_numeric_inputs]

In [ ]:
# 字符串输入

In [ ]:
# todo
